In [ ]:
import sqlite3

In [ ]:
import matplotlib.pyplot as plt

import cartopy.crs as ccrs
from cartopy.io import shapereader
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import cartopy.io.img_tiles as cimgt

from shapely.geometry import shape

from haversine import distance

In [ ]:
conn = sqlite3.connect('mwalks.sqlite')

In [ ]:
points = conn.execute('SELECT * FROM points ORDER BY route_id, point_id').fetchall()
print(len(points))

route_bounds = conn.execute('SELECT Min(lon), Min(lat), Max(lon), Max(lat) FROM points').fetchone()
print(bounds)

conn.close()

In [ ]:
def make_map(projection=ccrs.PlateCarree()):
    fig, ax = plt.subplots(figsize=(9, 13),
                           subplot_kw=dict(projection=projection))
    gl = ax.gridlines(draw_labels=True)
    gl.xlabels_top = gl.ylabels_right = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    return fig, ax

In [ ]:
request = cimgt.OSM()  # Open Street Map

pad_lat = 0.002
pad_lon = 0.002

lon_min, lat_min, lon_max, lat_max = route_bounds
extent = [lon_min - pad_lon, lon_max + pad_lon, lat_min - pad_lat, lat_max + pad_lat]

fig, ax = make_map(projection=request.crs)
ax.set_extent(extent)

img = ax.add_image(request, 14)

In [ ]:
print(points[:5])

In [ ]:
from collections import defaultdict

In [ ]:
from typing import NamedTuple

In [ ]:
import itertools

def pairwise(iterable):
    prev_item = None
    for current_item in iterable:
        yield (current_item, prev_item)
        prev_item = current_item
        
class Point(NamedTuple):
    point_id: int
    route_id: int
    dt: object
    lat: float
    lon: float
    ele: float

class Route:
    def __init__(self):
        self.points = []
        self.segments = None

    def add_point(self, pt):
        self.points.append(pt)

    @property
    def segments(self):
        if self.segments:
            return self.segments

In [ ]:
routes = defaultdict(Route)
for pt in [Point(*p) for p in points]:
    routes[pt.route_id].add_point(pt)
for route_id, rt in routes.items():
    print(f'Route {route_id}: {len(rt.points)} points')    
